In [11]:
import numpy as np
import scipy.sparse
from sklearn import svm, preprocessing, linear_model
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
import matplotlib.pyplot as plt
from collections import defaultdict

In [12]:
# utility functions 
def num_zero(lst):
    count = 0
    for x in lst:
        if x == 0:
            count += 1
    return count
    

def SVM_hyperparameter_finder(X, y, num_folds=5):
    crange = np.logspace(-2, 1, 5).tolist()
    param_grid = {'C': crange}
    svm = LinearSVC(penalty='l1', class_weight='balanced', C=C, dual=False)
    grid_search = GridSearchCV(svm, param_grid, cv=num_folds)
    grid_search.fit(X, y)
    print(grid_search.cv_results_)
    return grid_search.best_params_
    

In [10]:
#======SVM Trials=======
TYPE = "svm"
filenames = [("blood_type_B_no_filter_no_augmentation_X.npz", "blood_type_B_no_filter_no_augmentation_y.npy"),
             ("blood_type_Rh_chi2_balanced_augmentation_X.npz", "blood_type_Rh_chi2_balanced_augmentation_y.npy"),
             ("blood_type_Rh_chi2_no_augmentation_X.npz", "blood_type_Rh_chi2_no_augmentation_y.npy"),
             ("blood_type_A_chi2_no_augmentation_X.npz", "blood_type_A_chi2_no_augmentation_y.npy"),
             ("blood_type_B_chi2_no_augmentation_X.npz", "blood_type_B_chi2_no_augmentation_y.npy")]

for data, labels in filenames:
    print("Beginning work on another dataset...")
    X = scipy.sparse.load_npz(data)
    y = np.load(labels)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    dataset_name = data[:-6]
    f = open("trials/" + TYPE + "/" + dataset_name + "_" + TYPE + ".txt", "w")
    f.write("Details: LinearSVC with L1 Regularization\n")
    f.write("==========================================================\n")

    crange = np.logspace(-2, 1, 5).tolist()
    print("Training...")
    for C in crange:
        svc_test = LinearSVC(penalty='l1', class_weight='balanced', C=C, dual=False)
        svc_test.fit(X_train, y_train)
        score = svc_test.score(X_test, y_test)
        print("accuracy: %f\n" % score)
        f.write("accuracy: %f\n" % score)
        f.write("C: %f\n" % C)
        d = num_zero(list(svc_test.coef_[0]))
        num_coefficients = len(svc_test.coef_[0])
        percentage_zero = d / num_coefficients
        f.write("Number of zero coefficients: %f\n" % d)
        f.write("Percentage of zero coefficients: %f\n" % percentage_zero)
        f.write("==========================================================\n")
    print("Training completed for this dataset!")
    f.close()
print("Operations have completed and results have been written to disk!")

Beginning work on another dataset...
Training...
accuracy: 0.750000

accuracy: 0.937500

accuracy: 0.750000

accuracy: 0.750000

accuracy: 0.750000

Training completed for this dataset!
Beginning work on another dataset...
Training...
accuracy: 0.538462

accuracy: 0.923077

accuracy: 0.961538

accuracy: 0.961538

accuracy: 1.000000

Training completed for this dataset!
Beginning work on another dataset...
Training...
accuracy: 0.187500

accuracy: 0.750000

accuracy: 0.750000

accuracy: 0.750000

accuracy: 0.937500

Training completed for this dataset!
Beginning work on another dataset...
Training...
accuracy: 0.812500

accuracy: 1.000000

accuracy: 1.000000

accuracy: 1.000000

accuracy: 1.000000

Training completed for this dataset!
Beginning work on another dataset...
Training...
accuracy: 0.750000

accuracy: 0.937500

accuracy: 0.937500

accuracy: 0.937500

accuracy: 0.875000

Training completed for this dataset!
Operations have completed and results have been written to disk!


In [ ]:
#======Neural network Trials=======
TYPE = "neural_network"
LEARNING_RATE = .0001
BATCH_SIZE = 10
EPOCHS = 30
CROSS_VALIDATION_FOLDS = 5

isModelSummaryWritten = False
filenames = [("blood_type_B_no_filter_no_augmentation_X.npz", "blood_type_B_no_filter_no_augmentation_y.npy"),
             ("blood_type_Rh_chi2_balanced_augmentation_X.npz", "blood_type_Rh_chi2_balanced_augmentation_y.npy"),
             ("blood_type_Rh_chi2_no_augmentation_X.npz", "blood_type_Rh_chi2_no_augmentation_y.npy"),
             ("blood_type_A_chi2_no_augmentation_X.npz", "blood_type_A_chi2_no_augmentation_y.npy"),
             ("blood_type_B_chi2_no_augmentation_X.npz", "blood_type_B_chi2_no_augmentation_y.npy")]

def make_classifier():
    global isModelSummaryWritten
    
    model = Sequential()
    model.add(Dense(2, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=LEARNING_RATE), metrics=['accuracy'])
    
    if not isModelSummaryWritten:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
        isModelSummaryWritten = True
    return model

for data, labels in filenames:
    global isModelSummaryWritten
    isModelSummaryWritten = False
    
    print("Beginning work on another dataset...")
    X = scipy.sparse.load_npz(data)
    y = np.load(labels)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    dataset_name = data[:-6]
    
    f = open("trials/" + TYPE + "/" + dataset_name + "_" + TYPE + ".txt", "w")
    f.write("Details: No hidden layer--only an input and output layer with one relu and one sigmoid activation.\n")
    f.write("Learning Rate: %f\n" % LEARNING_RATE)
    f.write("Batch Size: %f\n" % BATCH_SIZE)
    f.write("==========================================================\n")
    
    classifier = KerasClassifier(build_fn = make_classifier, batch_size=BATCH_SIZE, epochs=EPOCHS)
    scores = cross_val_score(classifier, X, y, cv=CROSS_VALIDATION_FOLDS)
    print("Accuracy 5-fold: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std()))
    f.write("Accuracy 5-fold: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std()))
    f.close()
print("Operations have completed and results have been written to disk!")



In [13]:
#======Neural network Trials=======
TYPE = "neural_network_2"
LEARNING_RATE = .0001
BATCH_SIZE = 10
EPOCHS = 200
CROSS_VALIDATION_FOLDS = 5
DETAILS = "Details: One hidden layer. Very shallow but still quite different from a linear model\n"

isModelSummaryWritten = False
filenames = [("blood_type_B_no_filter_no_augmentation_X.npz", "blood_type_B_no_filter_no_augmentation_y.npy"),
             ("blood_type_Rh_chi2_balanced_augmentation_X.npz", "blood_type_Rh_chi2_balanced_augmentation_y.npy"),
             ("blood_type_Rh_chi2_no_augmentation_X.npz", "blood_type_Rh_chi2_no_augmentation_y.npy"),
             ("blood_type_A_chi2_no_augmentation_X.npz", "blood_type_A_chi2_no_augmentation_y.npy"),
             ("blood_type_B_chi2_no_augmentation_X.npz", "blood_type_B_chi2_no_augmentation_y.npy")]


def make_classifier():
    global isModelSummaryWritten
    
    model = Sequential()
    model.add(Dense(5, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(10, activation='sigmoid', kernel_initializer='he_normal'))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=LEARNING_RATE), metrics=['accuracy'])
    
    if not isModelSummaryWritten:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
        isModelSummaryWritten = True
    return model

for data, labels in filenames:
    global isModelSummaryWritten
    isModelSummaryWritten = False
    
    print("Beginning work on another dataset...")
    X = scipy.sparse.load_npz(data)
    y = np.load(labels)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    dataset_name = data[:-6]
    
    f = open("trials/" + TYPE + "/" + dataset_name + "_" + TYPE + ".txt", "w")
    f.write(DETAILS)
    f.write("Learning Rate: %f\n" % LEARNING_RATE)
    f.write("Batch Size: %f\n" % BATCH_SIZE)
    f.write("Number of Epochs: %f\n" % EPOCHS)
    f.write("==========================================================\n")
    
    classifier = KerasClassifier(build_fn = make_classifier, batch_size=BATCH_SIZE, epochs=EPOCHS)
    scores = cross_val_score(classifier, X, y, cv=CROSS_VALIDATION_FOLDS)
    print("Accuracy 5-fold: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std()))
    f.write("Accuracy 5-fold: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std()))
    f.close()
print("Operations have completed and results have been written to disk!")



Beginning work on another dataset...
Epoch 1/200
63/63 [==============================] - 46s 734ms/step - loss: 0.7605 - acc: 0.1905
Epoch 2/200
63/63 [==============================] - 36s 574ms/step - loss: 0.7550 - acc: 0.1905
Epoch 3/200
63/63 [==============================] - 33s 530ms/step - loss: 0.7534 - acc: 0.1905
Epoch 4/200
63/63 [==============================] - 39s 618ms/step - loss: 0.7519 - acc: 0.1905
Epoch 5/200
63/63 [==============================] - 36s 570ms/step - loss: 0.7505 - acc: 0.1905
Epoch 6/200
63/63 [==============================] - 34s 541ms/step - loss: 0.7490 - acc: 0.1905
Epoch 7/200
63/63 [==============================] - 35s 552ms/step - loss: 0.7474 - acc: 0.1905
Epoch 8/200
63/63 [==============================] - 35s 555ms/step - loss: 0.7459 - acc: 0.1905
Epoch 9/200
63/63 [==============================] - 34s 544ms/step - loss: 0.7443 - acc: 0.1905
Epoch 10/200
63/63 [==============================] - 35s 561ms/step - loss: 0.7429 - acc:

63/63 [==============================] - 31s 488ms/step - loss: 0.6517 - acc: 0.8095
Epoch 85/200
63/63 [==============================] - 32s 501ms/step - loss: 0.6506 - acc: 0.8095
Epoch 86/200
63/63 [==============================] - 32s 510ms/step - loss: 0.6497 - acc: 0.8095
Epoch 87/200
63/63 [==============================] - 32s 515ms/step - loss: 0.6488 - acc: 0.8095
Epoch 88/200
63/63 [==============================] - 31s 494ms/step - loss: 0.6478 - acc: 0.8095
Epoch 89/200
63/63 [==============================] - 32s 509ms/step - loss: 0.6468 - acc: 0.8095
Epoch 90/200
63/63 [==============================] - 31s 498ms/step - loss: 0.6458 - acc: 0.8095
Epoch 91/200
63/63 [==============================] - 32s 508ms/step - loss: 0.6449 - acc: 0.8095
Epoch 92/200
63/63 [==============================] - 33s 521ms/step - loss: 0.6438 - acc: 0.8095
Epoch 93/200
63/63 [==============================] - 32s 516ms/step - loss: 0.6428 - acc: 0.8095
Epoch 94/200
63/63 [=============

63/63 [==============================] - 15s 243ms/step - loss: 0.5833 - acc: 0.8095
Epoch 168/200
63/63 [==============================] - 16s 248ms/step - loss: 0.5827 - acc: 0.8095
Epoch 169/200
63/63 [==============================] - 15s 242ms/step - loss: 0.5821 - acc: 0.8095
Epoch 170/200
63/63 [==============================] - 15s 239ms/step - loss: 0.5814 - acc: 0.8095
Epoch 171/200
63/63 [==============================] - 15s 243ms/step - loss: 0.5808 - acc: 0.8095
Epoch 172/200
63/63 [==============================] - 15s 243ms/step - loss: 0.5802 - acc: 0.8095
Epoch 173/200
63/63 [==============================] - 15s 245ms/step - loss: 0.5796 - acc: 0.8095
Epoch 174/200
63/63 [==============================] - 15s 243ms/step - loss: 0.5790 - acc: 0.8095
Epoch 175/200
63/63 [==============================] - 15s 243ms/step - loss: 0.5783 - acc: 0.8095
Epoch 176/200
63/63 [==============================] - 15s 245ms/step - loss: 0.5777 - acc: 0.8095
Epoch 177/200
63/63 [===

63/63 [==============================] - 16s 250ms/step - loss: 0.4333 - acc: 0.8095
Epoch 50/200
63/63 [==============================] - 16s 246ms/step - loss: 0.4332 - acc: 0.8095
Epoch 51/200
63/63 [==============================] - 15s 241ms/step - loss: 0.4331 - acc: 0.8095
Epoch 52/200
63/63 [==============================] - 15s 241ms/step - loss: 0.4330 - acc: 0.8095
Epoch 53/200
63/63 [==============================] - 16s 247ms/step - loss: 0.4329 - acc: 0.8095
Epoch 54/200
63/63 [==============================] - 15s 242ms/step - loss: 0.4328 - acc: 0.8095
Epoch 55/200
63/63 [==============================] - 15s 240ms/step - loss: 0.4327 - acc: 0.8095
Epoch 56/200
63/63 [==============================] - 15s 245ms/step - loss: 0.4327 - acc: 0.8095
Epoch 57/200
63/63 [==============================] - 16s 247ms/step - loss: 0.4326 - acc: 0.8095
Epoch 58/200
63/63 [==============================] - 15s 241ms/step - loss: 0.4325 - acc: 0.8095
Epoch 59/200
63/63 [=============

63/63 [==============================] - 15s 241ms/step - loss: 0.4256 - acc: 0.8095
Epoch 133/200
63/63 [==============================] - 15s 243ms/step - loss: 0.4255 - acc: 0.8095
Epoch 134/200
63/63 [==============================] - 15s 238ms/step - loss: 0.4254 - acc: 0.8095
Epoch 135/200
63/63 [==============================] - 15s 240ms/step - loss: 0.4254 - acc: 0.8095
Epoch 136/200
63/63 [==============================] - 15s 237ms/step - loss: 0.4253 - acc: 0.8095
Epoch 137/200
63/63 [==============================] - 15s 244ms/step - loss: 0.4252 - acc: 0.8095
Epoch 138/200
63/63 [==============================] - 15s 240ms/step - loss: 0.4252 - acc: 0.8095
Epoch 139/200
63/63 [==============================] - 16s 248ms/step - loss: 0.4251 - acc: 0.8095
Epoch 140/200
63/63 [==============================] - 15s 243ms/step - loss: 0.4250 - acc: 0.8095
Epoch 141/200
63/63 [==============================] - 15s 244ms/step - loss: 0.4250 - acc: 0.8095
Epoch 142/200
63/63 [===

...

16/16 [==============================] - 0s 19ms/step
Epoch 1/200
63/63 [==============================] - 1s 14ms/step - loss: 0.6971 - acc: 0.4127
Epoch 2/200
63/63 [==============================] - 0s 2ms/step - loss: 0.5970 - acc: 0.8254
Epoch 3/200
63/63 [==============================] - 0s 2ms/step - loss: 0.5254 - acc: 0.8254
Epoch 4/200
63/63 [==============================] - 0s 2ms/step - loss: 0.4851 - acc: 0.8889
Epoch 5/200
63/63 [==============================] - 0s 2ms/step - loss: 0.4606 - acc: 0.9524
Epoch 6/200
63/63 [==============================] - 0s 2ms/step - loss: 0.4405 - acc: 0.9683
Epoch 7/200
63/63 [==============================] - 0s 2ms/step - loss: 0.4283 - acc: 0.9841
Epoch 8/200
63/63 [==============================] - 0s 2ms/step - loss: 0.4163 - acc: 0.9841
Epoch 9/200
63/63 [==============================] - 0s 2ms/step - loss: 0.4086 - acc: 0.9841
Epoch 10/200
63/63 [==============================] - 0s 2ms/step - loss: 0.4020 - acc: 0.9841
Epoc

63/63 [==============================] - 0s 2ms/step - loss: 0.2918 - acc: 0.9841
Epoch 87/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2914 - acc: 0.9841
Epoch 88/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2909 - acc: 0.9841
Epoch 89/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2905 - acc: 0.9841
Epoch 90/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2900 - acc: 0.9841
Epoch 91/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2895 - acc: 0.9841
Epoch 92/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2891 - acc: 0.9841
Epoch 93/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2887 - acc: 0.9841
Epoch 94/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2883 - acc: 0.9841
Epoch 95/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2879 - acc: 0.9841
Epoch 96/200
63/63 [==============================] - 0s 2ms/st

63/63 [==============================] - 0s 2ms/step - loss: 0.2621 - acc: 0.9365
Epoch 172/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2619 - acc: 0.9365
Epoch 173/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2617 - acc: 0.9365
Epoch 174/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2614 - acc: 0.9365
Epoch 175/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2612 - acc: 0.9365
Epoch 176/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2609 - acc: 0.9365
Epoch 177/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2606 - acc: 0.9365
Epoch 178/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2604 - acc: 0.9365
Epoch 179/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2601 - acc: 0.9365
Epoch 180/200
63/63 [==============================] - 0s 2ms/step - loss: 0.2598 - acc: 0.9365
Epoch 181/200
63/63 [==============================] -

64/64 [==============================] - 0s 2ms/step - loss: 0.1212 - acc: 1.0000
Epoch 57/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1208 - acc: 1.0000
Epoch 58/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1204 - acc: 1.0000
Epoch 59/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1200 - acc: 1.0000
Epoch 60/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1196 - acc: 1.0000
Epoch 61/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1192 - acc: 1.0000
Epoch 62/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1188 - acc: 1.0000
Epoch 63/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1184 - acc: 1.0000
Epoch 64/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1181 - acc: 1.0000
Epoch 65/200
64/64 [==============================] - 0s 2ms/step - loss: 0.1177 - acc: 1.0000
Epoch 66/200
64/64 [==============================] - 0s 2ms/st

64/64 [==============================] - 0s 2ms/step - loss: 0.0947 - acc: 1.0000
Epoch 142/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0944 - acc: 1.0000
Epoch 143/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0942 - acc: 1.0000
Epoch 144/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0939 - acc: 1.0000
Epoch 145/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0937 - acc: 1.0000
Epoch 146/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0934 - acc: 1.0000
Epoch 147/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0932 - acc: 1.0000
Epoch 148/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0929 - acc: 1.0000
Epoch 149/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0927 - acc: 1.0000
Epoch 150/200
64/64 [==============================] - 0s 2ms/step - loss: 0.0925 - acc: 1.0000
Epoch 151/200
64/64 [==============================] -